In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')  # enable import from src/

In [32]:
from pathlib import Path
import torch

from torcheval.metrics import BinaryF1Score, BinaryAccuracy
from torch.utils.data.sampler import SubsetRandomSampler
from dgl.dataloading import GraphDataLoader

from src.net import InstanceGCN
from src.utils import load_from_wandb
from src.dataset import JobFeasibilityDataset, SatelliteFeasibilityDataset

In [30]:
net = InstanceGCN(2,)
net = load_from_wandb(net, 'gebp7q0y', 'sat-gnn', 'model_best')

# only last instance
data = JobFeasibilityDataset('../data/raw/97_9.jl')
sampler = SubsetRandomSampler(torch.arange(8000, len(data)))

dl = GraphDataLoader(data, sampler=sampler, batch_size=2**5)

acc = BinaryAccuracy()
f1 = BinaryF1Score()

net.eval()
with torch.no_grad():
    for X, y in dl:
        y_hat = torch.sigmoid(net(X)).squeeze(1)

        acc.update(y_hat, y)
        f1.update(y_hat, y)
acc.compute(), f1.compute()

(tensor(0.8300), tensor(0.8172))

In [34]:
net = InstanceGCN(2,)
net = load_from_wandb(net, '12arvgxb', 'sat-gnn', 'model_best')

# only last instance
data = SatelliteFeasibilityDataset(Path('../data/raw/').glob('97_9*.jl'))
sampler = SubsetRandomSampler(torch.arange(19*1000, len(data)))

dl = GraphDataLoader(data, sampler=sampler, batch_size=2**5)

acc = BinaryAccuracy()
f1 = BinaryF1Score()

net.eval()
with torch.no_grad():
    for X, y in dl:
        y_hat = torch.sigmoid(net(X)).squeeze(1)

        acc.update(y_hat, y)
        f1.update(y_hat, y)
acc.compute(), f1.compute()

(tensor(0.7515), tensor(0.7956))